From a slack message by L Grey in the `servicex-cms` chat.

In [1]:
from pathlib import Path
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from dask.distributed import Client
import dask
import dask_awkward
import awkward as ak
import hist.dask as hda

def extract_pushdown(coll):
    hlg_sorted = coll.dask._toposort_layers()
    pushdown_deps = []
    for key in hlg_sorted:
        annotations = coll.dask.layers[key].annotations
        if annotations is not None and "pushdown" in annotations:
            #print(key, coll.dask.layers[key].annotations)
            pushdown_deps = [key] + pushdown_deps
    for dep in pushdown_deps:
        layer = coll.dask.layers[dep]
        fcn = list(layer.dsk.values())[0][0]
        if isinstance(layer, dask_awkward.layers.AwkwardBlockwiseLayer) and not isinstance(layer, dask_awkward.layers.AwkwardInputLayer):
            print(layer.dsk)
            # print(dir(layer))
            print(list(layer.keys()))
            print(dep, fcn.fn)
            # print(dir(fcn))
            # print(fcn.arg_repackers[0])
            print("")
        else:
            print(dep, fcn)

if __name__ == "__main__":
    #client = Client()


    dask.config.set({"awkward.optimization.enabled": True, "awkward.raise-failed-meta": True, "awkward.optimization.on-fail": "raise"})

    with dask.annotate(pushdown="servicex"):
        events = NanoEventsFactory.from_root(
            {
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
                "nano_dy.root": "Events",
            },
            metadata={"dataset": "nano_dy"},
            schemaclass=NanoAODSchema,
        ).events()

        mask = events.Muon.pt > 30
        events = events[ak.any(mask, axis=1)]
        
    myhist = hda.Hist.new.Regular(50, 0, 2.5, name="abseta").Double()

    myhist.fill(abseta=abs(events.Muon.eta))

    extract_pushdown(myhist)

c:\Users\gordo\Code\iris-hep\awkward-20-testing\.venv\Lib\site-packages\coffea\nanoevents\methods\candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


{'getitem-3e3d3493a7a807d06dd9ad3d451f1403': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x000001F4FEFD7D90>, '__dask_blockwise__0', '__dask_blockwise__1')}
[('getitem-3e3d3493a7a807d06dd9ad3d451f1403', 0)]
getitem-3e3d3493a7a807d06dd9ad3d451f1403 <built-in function getitem>

{'any-d2eb1bfffdf8a8cdbcd69731a5bdfc19': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x000001F4A21CE290>, '__dask_blockwise__0')}
[('any-d2eb1bfffdf8a8cdbcd69731a5bdfc19', 0)]
any-d2eb1bfffdf8a8cdbcd69731a5bdfc19 <function any at 0x000001F4F4B53EC0>

{'greater-d13a053b2d57dea2ebe0c3fb0dcf925e': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x000001F4FF09AA50>, '__dask_blockwise__0', '__dask_blockwise__1')}
[('greater-d13a053b2d57dea2ebe0c3fb0dcf925e', 0)]
greater-d13a053b2d57dea2ebe0c3fb0dcf925e <ufunc 'greater'>

{'pt-c9313c680c4c7960491a42eefa101b22': (<dask_awkward.lib.core.ArgsKwargsPackedFunction object at 0x000001F4FFF2F290>, '__dask_blockwise__0', '__dask_blockwise

c:\Users\gordo\Code\iris-hep\awkward-20-testing\.venv\Lib\site-packages\coffea\nanoevents\schemas\nanoaod.py:243: RuntimeWarning: Missing cross-reference index for FatJet_genJetAK8Idx => GenJetAK8
  warnings.warn(
